### Adding biographies
In this notebook I seek to carry out the same chatGPT experiments with less examples and more clear definitions first. Then I'll compare against previous experiments and finally add the biograhpies as context, to see its influence. I might also try embedding variations. <br>
Exact list of planned baseline comparison:
1. Embedding of post baseline
2. Embedding of post with simple instruction (no CoT)
3. Embedding of post with instruction & example (no CoT)
4. Embedding of post with instruction, example and CoT
5. chatGPT completions all of the above instead of embeddings
6. All of the above with Biographies

I'll do this with the 200 double checked posts or a similarly sized sample.<br>
For flexibility with the prompts, I probably do not want to do this in a modular fashion

In [29]:
import pandas as pd
import sys
import openai
from helpers.GPTclassifier import gptclassifier
openai.api_key = ""

# import instagram profiles
profiles = pd.read_pickle("data/df_profiles.pkl")

# import posts
sys.path.append('../7-Self-Labelled-Data')
df = pd.read_pickle("../7-Self-Labelled-Data/data/annotated_test_explantions_gpt3.pkl")

In [32]:
# remove the less represented influencers; maintains 70 % of the data
df = df[df["username"].isin(df['username'].value_counts().nlargest(20).index)]

In [37]:
df_sampled = df.groupby('username', group_keys=False).apply(lambda x: x.sample(min(len(x), 10)))

In [52]:
df_sampled = df_sampled.merge(profiles[['username', 'full_name', 'edge_followed_by', 'biography']], on='username', how='left')

In [53]:
df_sampled

,username,caption,caption_hashtags,tagged_users,subscriber_count,ct_score,counts_disabled,likes,likes_expected,comments,...,dt_year_mon,predicted_disclosure,year,caption_len,label,caption_2,explanations-gpt3,full_name,edge_followed_by,biography
0,ab_bowen,New @gymking training vests 👀😍,[],[gymking],1632588,-4.131148,False,1818,7539,12,...,2022/07,True,2022,5,True,New USR_gymking training vests 👀😍,Key indicators: '@gymking'.\nReasoning: This p...,ALEX BOWEN,1651545,REPRESENTED BY FREDDY@WHITELABELMGMT.CO.UK\nFI...
1,ab_bowen,So tomorrow is the big day! 🚀🕺🏼 back with the ...,[],[levels],1632601,-12.436836,False,829,10504,18,...,2022/07,True,2022,24,False,So tomorrow is the big day! 🚀🕺🏼 back with the ...,"Key indicators: '@levels.birmingham', 'click t...",ALEX BOWEN,1651545,REPRESENTED BY FREDDY@WHITELABELMGMT.CO.UK\nFI...
2,ab_bowen,New Gym tees dropping this Friday! @exemptsociety,[],[exemptsociety],1605729,-5.186678,False,2272,11805,10,...,2022/05,False,2022,7,False,New Gym tees dropping this Friday! USR_exempts...,Key indicators: '@exemptsociety'.\nReasoning: ...,ALEX BOWEN,1651545,REPRESENTED BY FREDDY@WHITELABELMGMT.CO.UK\nFI...
3,ab_bowen,There’s a lot going on in the Bowen household ...,[],[Toppikman],1605034,-3.483729,False,1122,3922,15,...,2022/05,True,2022,63,False,There’s a lot going on in the Bowen household ...,Key indicators: '@Toppikman'.\nReasoning: The ...,ALEX BOWEN,1651545,REPRESENTED BY FREDDY@WHITELABELMGMT.CO.UK\nFI...
4,ab_bowen,Online coaching launching real soon! Click the...,[],[gymking],1628052,-5.163934,False,1454,7539,10,...,2022/09,True,2022,20,False,Online coaching launching real soon! Click the...,Key indicators: '@gymking'.\nReasoning: The po...,ALEX BOWEN,1651545,REPRESENTED BY FREDDY@WHITELABELMGMT.CO.UK\nFI...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,wildbird,A comfortable and safe fit is key when it come...,[],[wildbird],328587,-2.100437,False,455,935,3,...,2022/02,True,2022,82,False,A comfortable and safe fit is key when it come...,Key indicators: None.\nReasoning: The post is ...,WILDBIRD®,302495,Happy place for you & your little bird!\nRing ...
196,wildbird,Minis are made for your little mini me! One of...,"[minisling, easterbasket, eastergiftideas, sli...",[],328266,1.372236,False,2202,1408,32,...,2022/03,True,2022,59,False,Minis are made for your little mini me! One of...,"Key indicators: '#easterbasket', '#eastergifti...",WILDBIRD®,302495,Happy place for you & your little bird!\nRing ...
197,wildbird,Oh hi! Wouldn't it be nice to have $150 to spe...,"[giveaway, babymusthaves, babyessentials]","[wildbirdbaby, plumandsparrow, wildbirdbaby]",328631,-2.264151,False,412,933,12,...,2022/03,True,2022,32,True,Oh hi! Wouldn't it be nice to have $150 to spe...,"Key indicators: '#giveaway','@wildbirdbaby', '...",WILDBIRD®,302495,Happy place for you & your little bird!\nRing ...
198,wildbird,Quickest. Prettiest. Most versatile. A ringsli...,"[ringsling, babywearing, linensling, babyessen...",[],328713,2.821317,False,2488,930,212,...,2022/03,True,2022,57,False,Quickest. Prettiest. Most versatile. A ringsli...,"Key indicators: '@wildbird', 'gift on of you $...",WILDBIRD®,302495,Happy place for you & your little bird!\nRing ...


In [19]:
#df_filtered = 
#df.groupby('username').count(sort ascending)#.filter(lambda x: len(x) > 50)
df.groupby('username').count()#.sort_values(ascending=True)

,caption,caption_hashtags,tagged_users,subscriber_count,ct_score,counts_disabled,likes,likes_expected,comments,comments_expected,...,caption_task,shortcode,followers,dt_year_mon,predicted_disclosure,year,caption_len,label,caption_2,explanations-gpt3
username,,,,,,,,,,,,,,,,,,,,,
ab_bowen,24,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24
alecmonopoly,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
allhailkingsteph,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
amandacerny,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
beberexha,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tysondayley,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
wesleymilliondollarvirgin,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
weworewhat,66,66,66,66,66,66,66,66,66,66,...,66,66,66,66,66,66,66,66,66,66


In [7]:
df_filtered

,username,caption,caption_hashtags,tagged_users,subscriber_count,ct_score,counts_disabled,likes,likes_expected,comments,...,caption_task,shortcode,followers,dt_year_mon,predicted_disclosure,year,caption_len,label,caption_2,explanations-gpt3
558102,wildbird,🌟💖 LET’S CELEBRATE WILDBIRD BABY! WIN THE WHOL...,"[babybrand, babyessentials, linenbedding, line...","[brenden_graczak, wildbird, wildbirdbaby]",328731,14.205208,False,4135,933,9502,...,🌟 💖 LET ’ S CELEBRATE WILDBIRD BABY ! WIN THE ...,CZ7Pqk8pGHX,326541,2022/02,True,2022,113,True,🌟💖 LET’S CELEBRATE WILDBIRD BABY! WIN THE WHOL...,"Key indicators: 'Follow @wildbirdbaby', 'Tag f..."
778440,thestilettomeup,Back at the office 💛\n\nYellow is my absolute ...,[],"[byegreis, dior, hermes]",513026,1.148215,False,6312,5486,87,...,Back at the office 💛 Yellow is my absolute fav...,Cb8NNmuJ8I6,519012,2022/04,True,2022,21,True,Back at the office 💛\n\nYellow is my absolute ...,"Key indicators: 'Outfit @byegreis', 'Bag @dior..."
778263,thestilettomeup,"I don’t come out often, but when I do….. 😈 \n\...",[],[byegreis],517781,-50.770115,True,0,4348,87,...,"I don ’ t come out often , but when I do … .. ...",CgK-UTiJfYa,519012,2022/07,False,2022,17,True,"I don’t come out often, but when I do….. 😈 \n\...",Key indicators: 'Suit @byegreis'.\nReasoning: ...
218532,weworewhat,first meeting of the day 🥰🐰,[],[],2845728,-1.228367,False,11006,13483,42,...,first meeting of the day 🥰 🐰,CakPLuAuf3H,3009747,2022/03,False,2022,6,False,first meeting of the day 🥰🐰,Reasoning: No indication of promoted product o...
224042,weworewhat,We spent over a year developing these new styl...,[],[],2856274,-2.295844,False,5200,11968,46,...,We spent over a year developing these new styl...,CbaGAyIuxC4,3009747,2022/03,True,2022,68,True,We spent over a year developing these new styl...,Key indicators: 'new slit scoop sports bra and...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558121,wildbird,Home is where the heart is ❤️\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\nDo y...,"[babywearing, ringsling, babynecessities, baby...",[little_fawn_],329218,-2.577128,False,371,933,5,...,Home is where the heart is ❤ ️ ⠀ ⠀ ⠀ Do you ha...,CZKmxNuPUai,326541,2022/01,True,2022,47,True,Home is where the heart is ❤️\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\nDo y...,"Key indicators: '#babywearing', '#ringsling', ..."
99832,chrisjerichofozzy,Heading into the weekend like… #JAS,[JAS],[],4056313,1.441863,False,15276,10565,113,...,Heading into the weekend like … #JAS,CcZGQj6s_mu,4217971,2022/04,False,2022,6,False,Heading into the weekend like… HT_JAS,Key indicators: None.\nReasoning: The post lac...
354641,claudiajordan,#Dallas I need you all to come out and support...,[Dallas],"[keepingfamiliesconnected, iamelizabethdemoraes]",599733,-5.406593,False,259,1403,14,...,#Dallas I need you all to come out and support...,CdJwRvKLKDu,606986,2022/05,True,2022,35,False,HT_Dallas I need you all to come out and supp...,Key indicators: '@keepingfamiliesconnected'.\n...
218529,weworewhat,The @maisonmargiela bag I should have at the a...,[],[maisonmargiela],2846124,1.292571,False,17418,13414,33,...,The @maisonmargiela bag I should have at the a...,CapbIQnOX4i,3009747,2022/03,True,2022,21,True,The USR_maisonmargiela bag I should have at th...,Key indicators: '@maisonmargiela'.\nReasoning:...


In [8]:
df

,username,caption,caption_hashtags,tagged_users,subscriber_count,ct_score,counts_disabled,likes,likes_expected,comments,...,caption_task,shortcode,followers,dt_year_mon,predicted_disclosure,year,caption_len,label,caption_2,explanations-gpt3
43427,therock,"Ahead of TONIGHT’s #YoungRock season premiere,...","[YoungRock, 1, YOUNGROCK]","[8PM, NBC, SevenBucksProd]",303804301,-2.267353,False,267341,605276,1805,...,Ahead of TONIGHT ’ s #YoungRock season premier...,CbJCKIvFqMI,338004780,2022/03,False,2022,37,False,Ahead of TONIGHT’s HT_YoungRock season premier...,"Key indicators: '#YoungRock', 'NBC', '@SevenBu..."
673382,thisishowilook,BTS at @priscavera . I talked about how tough...,[],[priscavera],328970,-2405.800000,True,0,11967,5,...,BTS at @priscavera . I talked about how tough ...,Cixz_Z2OkOM,328914,2022/09,True,2022,92,False,BTS at USR_priscavera . I talked about how tou...,Key indicators: '@priscavera'.\nReasoning: The...
558102,wildbird,🌟💖 LET’S CELEBRATE WILDBIRD BABY! WIN THE WHOL...,"[babybrand, babyessentials, linenbedding, line...","[brenden_graczak, wildbird, wildbirdbaby]",328731,14.205208,False,4135,933,9502,...,🌟 💖 LET ’ S CELEBRATE WILDBIRD BABY ! WIN THE ...,CZ7Pqk8pGHX,326541,2022/02,True,2022,113,True,🌟💖 LET’S CELEBRATE WILDBIRD BABY! WIN THE WHOL...,"Key indicators: 'Follow @wildbirdbaby', 'Tag f..."
733858,thepocketqueen,2 days ago ♠️𝗣𝗼𝗰𝗸𝗲𝘁 𝗤𝘂𝗲𝗲𝗻 & 𝗧𝗵𝗲 𝗥𝗼𝘆𝗮𝗹 𝗙𝗹𝘂𝘀𝗵 ♥️...,[TinyDeskContest],"[nprmusic, ryckjane, iammattrose, farahstop]",496690,-158.868852,True,0,9519,61,...,2 days ago ♠ ️𝗣𝗼𝗰𝗸𝗲𝘁 𝗤𝘂𝗲𝗲𝗻 & 𝗧𝗵𝗲 𝗥𝗼𝘆𝗮𝗹 𝗙𝗹𝘂𝘀𝗵 ♥...,Ce3vA9ILORa,514148,2022/06,True,2022,62,False,2 days ago ♠️𝗣𝗼𝗰𝗸𝗲𝘁 𝗤𝘂𝗲𝗲𝗻 & 𝗧𝗵𝗲 𝗥𝗼𝘆𝗮𝗹 𝗙𝗹𝘂𝘀𝗵 ♥️...,Key indicators: none.\nReasoning: There is no ...
138743,cleopatraa_duess,Cook-off w/ friends dropped @6pm click the lin...,[],[6pm],543238,-7.237312,False,1384,10033,15,...,Cook-off w / friends dropped @6pm click the li...,CavbXqMliQl,524409,2022/03,True,2022,22,False,Cook-off w/ friends dropped USR_6pm click the ...,Key indicators: none.\nReasoning: There is no ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311981,alecmonopoly,💲Dubai Mall Art Show March 24th💲I will be pain...,[],[edengallery],1407006,-86.355030,True,0,14468,169,...,💲 Dubai Mall Art Show March 24th 💲 I will be p...,CbXyt5hr72z,1421631,2022/03,True,2022,17,False,💲Dubai Mall Art Show March 24th💲I will be pain...,Key indicators: '@edengallery'.\nReasoning: Al...
473421,nadia_mejia,Constantly being on the go and in different ti...,"[Ad, CleanandClearPartner]",[cleanandclear],290652,-171.818182,True,0,3724,22,...,Constantly being on the go and in different ti...,ChZ9ZKhr-Mk,289601,2022/08,False,2022,78,True,Constantly being on the go and in different ti...,Key indicators: 'breakout besties @cleanandcle...
218529,weworewhat,The @maisonmargiela bag I should have at the a...,[],[maisonmargiela],2846124,1.292571,False,17418,13414,33,...,The @maisonmargiela bag I should have at the a...,CapbIQnOX4i,3009747,2022/03,True,2022,21,True,The USR_maisonmargiela bag I should have at th...,Key indicators: '@maisonmargiela'.\nReasoning:...
462947,questlove,The is hands down post 👏🏾 of 👏🏾the👏🏾year @whit...,[],[whitehouse],2635670,5.413212,False,50169,9348,1538,...,The is hands down post 👏🏾 of 👏🏾 the 👏🏾 year @w...,ChwE2YIOjRW,2652823,2022/08,True,2022,9,False,The is hands down post 👏🏾 of 👏🏾the👏🏾year USR_w...,Key indicators: none.\nReasoning: No direct or...


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
